In [1]:
from UTILS.network import TcpServerP2P, TcpClientP2P
import threading, time
import numpy as np
ipport = ('127.0.0.1', 25455)
server = TcpServerP2P(ipport, obj='pickle')
client = TcpClientP2P(ipport, obj='pickle')
def server_fn():
    time.sleep(1)
    data = server.wait_next_dgram()
    print('[server]: recv', data)
    server.reply_last_client(np.array([4,5,6]))
    print('[server]: done reply')

def noreply_server_fn():
    time.sleep(1)
    data = server.wait_multi_dgrams()
    print('[server]: recv', data)


def client_fn():
    print('[client]: sending np.array([1,2,3])')
    rep = client.send_and_wait_reply(np.array([1,2,3]))
    print('[client]: recv', rep)

def nowait_client_fn():
    print('[client]: sending np.array([1,2,3])')
    rep = client.send_dgram_to_target(np.array([1,2,3]))
    print('[client]: sending np.array([4,5,6])')
    rep = client.send_dgram_to_target(np.array([4,5,6]))
    
thread_hi = threading.Thread(target=noreply_server_fn)
thread_hello = threading.Thread(target=nowait_client_fn)
# 启动线程
thread_hi.start()
thread_hello.start()


[client]: sending np.array([1,2,3])
[client]: sending np.array([4,5,6])
[server]: recv [array([1, 2, 3]), array([4, 5, 6])]


In [11]:
import torch
import numpy as np
from torch.distributions.categorical import Categorical

def _get_act_log_probs(distribution, action):
    return distribution.log_prob(action.squeeze(-1)).unsqueeze(-1)

 
hyper_act_logits = torch.Tensor([0.32, 0.75])
logits_agent_cluster = torch.Tensor([0.7, 0.2, 0.1])
test_mode = False
eval_mode = False

h_dist = Categorical(logits = hyper_act_logits)
print('h两个动作的概率:', h_dist.probs)
if not test_mode:  h_act = h_dist.sample() if not eval_mode else eval_actions
else:              h_act = torch.argmax(h_dist.probs)
print('h动作为(1为采样, 0为贪婪最大):', h_act.item())
# SampleLogProb = torch.log(h_dist.probs[..., 1])
SampleLogProb = _get_act_log_probs(h_dist, torch.ones_like(h_act))
hActLogProbsRef = _get_act_log_probs(h_dist, h_act)
print('选定h动作的概率:', torch.exp(hActLogProbsRef))


act_dist = Categorical(logits = logits_agent_cluster)
print('act动作概率:', act_dist.probs)
act_sample = act_dist.sample() if not eval_mode else eval_actions
act_argmax = torch.argmax(act_dist.probs)

act = torch.where(h_act==1, act_sample, act_argmax) # h_act: shape=($n_thread, $n_agent)

################# 至此，已经选出了最后的动作 ################

sel_argmax = (act_argmax==act)
print('act动作选择', act)

actLogProbs01 = _get_act_log_probs(act_dist, act) # the policy gradient loss will feedback from here
print('选定act动作选择的概率:', torch.exp(actLogProbs01))


actLogProbs_notargmax = actLogProbs01 + SampleLogProb
actLogProbs_argmax = torch.log(torch.exp(actLogProbs01 + SampleLogProb) + (1-torch.exp(SampleLogProb)))
actLogProbs = torch.where(sel_argmax, actLogProbs_argmax, actLogProbs_notargmax)
print('actProbs:', torch.exp(actLogProbs))

'''
    h两个动作的概率: tensor([0.3941, 0.6059])
    h动作为(1为采样, 0为贪婪最大): 0
    选定h动作的概率: tensor([0.3941])
    act动作概率: tensor([0.4640, 0.2814, 0.2546])
    act动作选择 tensor(0)
    选定act动作选择的概率: tensor([0.4640])
    actProbs: tensor([0.6752])
'''

h两个动作的概率: tensor([0.3941, 0.6059])
h动作为(1为采样, 0为贪婪最大): 1
选定h动作的概率: tensor([0.6059])
act动作概率: tensor([0.4640, 0.2814, 0.2546])
act动作选择 tensor(2)
选定act动作选择的概率: tensor([0.2546])
actProbs: tensor([0.1543])


'\n    h两个动作的概率: tensor([0.3941, 0.6059])\n    h动作为(1为采样, 0为贪婪最大): 0\n    选定h动作的概率: tensor([0.3941])\n    act动作概率: tensor([0.4640, 0.2814, 0.2546])\n    act动作选择 tensor(0)\n    选定act动作选择的概率: tensor([0.4640])\n    actProbs: tensor([0.6752])\n'

In [139]:
import torch
import numpy as np
from torch.distributions.categorical import Categorical

def _get_act_log_probs(distribution, action):
    return distribution.log_prob(action.squeeze(-1)).unsqueeze(-1)

res = []
for i in range(10000):

    hyper_act_logits = torch.Tensor([0.5, 0.5])
    logits_agent_cluster = torch.Tensor([0.7, 0.2, 0.1])
    test_mode = False
    eval_mode = False

    h_dist = Categorical(logits = hyper_act_logits)
    # print('h_dist:', h_dist.probs)
    if not test_mode:  h_act = h_dist.sample() if not eval_mode else eval_actions
    else:              h_act = torch.argmax(h_dist.probs)
    # print('h_act(1为采样, 0为最大):', h_act)

    hActLogProbs = _get_act_log_probs(h_dist, h_act)
    # print('hActProbs:', torch.exp(hActLogProbs))


    act_dist = Categorical(logits = logits_agent_cluster)
    # print('act_dist:', act_dist.probs)
    act_sample = act_dist.sample() if not eval_mode else eval_actions
    act_argmax = torch.argmax(act_dist.probs)

    act = torch.where(h_act==1, act_sample, act_argmax) # h_act: shape=($n_thread, $n_agent)
    sel_argmax = (act_argmax==act)
    # print('act:', act)

    actLogProbs01 = _get_act_log_probs(act_dist, act) # the policy gradient loss will feedback from here
    # print('actProbs01:', torch.exp(actLogProbs01))


    actLogProbs_notargmax = actLogProbs01 + hActLogProbs
    actLogProbs_argmax = torch.log(torch.exp(actLogProbs01 + hActLogProbs) + (1-torch.exp(hActLogProbs)))
    actLogProbs = torch.where(sel_argmax, actLogProbs_argmax, actLogProbs_notargmax)
    # print('actProbs:', torch.exp(actLogProbs))
    res.append(act.item())
res = np.array(res)
len_res = len(res)
print(sum(res==0)/len_res)
print(sum(res==1)/len_res)
print(sum(res==2)/len_res)

0.7296
0.1399
0.1305


client.send_targeted_dgram('ddd')

1.0

In [ ]:
import torch
import numpy as np
from torch.distributions.categorical import Categorical

def _get_act_log_probs(distribution, action):
    return distribution.log_prob(action.squeeze(-1)).unsqueeze(-1)

 
hyper_act_logits = torch.Tensor([0.5, 0.5])
logits_agent_cluster = torch.Tensor([0.7, 0.2, 0.1])
test_mode = False
eval_mode = False

h_dist = Categorical(logits = hyper_act_logits)
if not test_mode:  h_act = h_dist.sample() if not eval_mode else eval_actions
else:              h_act = torch.argmax(h_dist.probs)
hActLogProbs = _get_act_log_probs(h_dist, h_act)

act_dist = Categorical(logits = logits_agent_cluster)
act_sample = act_dist.sample() if not eval_mode else eval_actions
act_argmax = torch.argmax(act_dist.probs)

act = torch.where(h_act==1, act_sample, act_argmax) # h_act: shape=($n_thread, $n_agent)
sel_argmax = (act_argmax==act)

actLogProbs01 = _get_act_log_probs(act_dist, act) # the policy gradient loss will feedback from here

actLogProbs_notargmax = actLogProbs01 + hActLogProbs
actLogProbs_argmax = torch.log(torch.exp(actLogProbs01 + hActLogProbs) + (1-torch.exp(hActLogProbs)))
actLogProbs = torch.where(sel_argmax, actLogProbs_argmax, actLogProbs_notargmax)


In [ ]:
import numpy as np
from UTILS.tensor_ops import _2tensor, cfg
cfg.device_ = 'cuda:0'
cfg.use_float64_ = False
cfg.init = True

input_dim = 10
my_data_sample_x = np.random.rand(1000,10)*10 - 1 # random from -1 to +1
my_data_sample_y = my_data_sample_x.mean(-1)**2 + my_data_sample_x.mean(-1)*(-2) + 1

# solve the weight inital problem
vae_mod = VanillaVAE(input_dim=input_dim, latent_dim=16, hidden_dim=32, degenerate2ae=False)
my_data_sample_x = _2tensor(my_data_sample_x)
vae_mod = _2tensor(vae_mod)
from torch.optim import Adam
optimizer = Adam(vae_mod.parameters(), lr=3e-3)

print("Start training VAE...")
vae_mod.train()

for epoch in range(5000):
    overall_loss = 0
    x = my_data_sample_x

    optimizer.zero_grad()

    #  [self.decode(z), input, mu, log_var]    #  x_hat, mean, log_var
    x_hat, x_origin, mean, log_var = vae_mod(x) # model(x)
    error = ( torch.abs(x_hat)-torch.abs(x) )/( torch.abs(x)+1e-9 )
    std_, mean_ = torch.std_mean(error, unbiased=False)
    print('\test error mean %.2f and std %.2f'%(mean_.item(), std_.item()) )


    lossdict = vae_mod.loss_function(x=x, x_hat=x_hat, mean=mean, log_var=log_var, kld_loss_weight=1, recons_loss_weight=1)
    loss = lossdict['loss']
    overall_loss += loss.item()

    loss.backward()
    optimizer.step()
    # 'Reconstruction_Loss':recons_loss.detach(), 'KLD'
    print("\tEpoch", epoch + 1, "complete!", "\tReconstruction_Loss: ", lossdict['Reconstruction_Loss'].item(), "\tKLD: ", lossdict['KLD'].item())

print("Finish!!")

In [ ]:
import torch
# from models import BaseVAE
from torch import nn
from torch.nn import functional as F
from torch import nn, Tensor
# from abc import abstractmethod


class VanillaVAE(nn.Module):

    def __init__(self,
                 input_dim: int,
                 latent_dim: int,
                 hidden_dim: int,
                 degenerate2ae = False,
                 **kwargs) -> None:
        super(VanillaVAE, self).__init__()
        self.training = True

        # Part 1, encoder
        self.FC_input = nn.Linear(input_dim, hidden_dim)
        self.FC_input2 = nn.Linear(hidden_dim, hidden_dim)
        self.FC_mean  = nn.Linear(hidden_dim, latent_dim)
        self.FC_var   = nn.Linear (hidden_dim, latent_dim)
        
        self.LeakyReLU = nn.LeakyReLU(0.2)
        
        # Part 2, decoder
        self.FC_hidden = nn.Linear(latent_dim, hidden_dim)
        self.FC_hidden2 = nn.Linear(hidden_dim, hidden_dim)
        self.FC_output = nn.Linear(hidden_dim, input_dim)
        
        self.LeakyReLU = nn.LeakyReLU(0.2)
        
        self.degenerate2ae = degenerate2ae

    def encode(self, input: Tensor):
        """
        Encodes the input by passing through the encoder network
        and returns the latent codes.
        :param input: (Tensor) Input tensor to encoder
        :return: (Tensor) List of latent codes
        """
        h_       = self.LeakyReLU(self.FC_input(input))
        h_       = self.LeakyReLU(self.FC_input2(h_))
        mean     = self.FC_mean(h_)
        log_var  = self.FC_var(h_) 

        return mean, log_var

    def decode(self, z: Tensor) -> Tensor:
        """
        Maps the given latent codes
        onto the image space.
        :param z: (Tensor) [B x D]
        :return: (Tensor) [B x C x H x W]
        """
        h     = self.LeakyReLU(self.FC_hidden(z))
        h     = self.LeakyReLU(self.FC_hidden2(h))
        x_hat = self.FC_output(h)

        return x_hat

    def reparameterize(self, mu: Tensor, logvar: Tensor) -> Tensor:
        """
        Reparameterization trick to sample from N(mu, var) from
        N(0,1).
        :param mu: (Tensor) Mean of the latent Gaussian [B x D]
        :param logvar: (Tensor) Standard deviation of the latent Gaussian [B x D]
        :return: (Tensor) [B x D]
        """
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return eps * std + mu

    def forward(self, input: Tensor, **kwargs):
        mu, log_var = self.encode(input)
        if not self.degenerate2ae:
            z = self.reparameterize(mu, log_var)
        else:
            z = mu
        return  [self.decode(z), input, mu, log_var]    #  x_hat, mean, log_var

    def loss_function(self,
                      x, x_hat, mean, log_var, kld_loss_weight, recons_loss_weight,
                      **kwargs) -> dict:
        """
        Computes the VAE loss function.
        KL(N(\mu, \sigma), N(0, 1)) = \log \frac{1}{\sigma} + \frac{\sigma^2 + \mu^2}{2} - \frac{1}{2}
        :param args:
        :param kwargs:
        :return:
        """
        recons = x_hat
        input = x
        mu = mean
        log_var = log_var

        # kld_weight = kwargs['M_N'] # Account for the minibatch samples from the dataset
        recons_loss = F.mse_loss(recons, input)

        # 计算高斯分布和标准正态分布的KL散度
        if not self.degenerate2ae:
            kld_loss = torch.mean(-0.5 * torch.sum(1 + log_var - mu ** 2 - log_var.exp(), dim = 1), dim = 0)
            loss = recons_loss*recons_loss_weight + kld_loss*kld_loss_weight
        else:
            kld_loss = torch.zeros_like(recons_loss)
            loss = recons_loss

        return {'loss': loss, 'Reconstruction_Loss':recons_loss.detach(), 'KLD':-kld_loss.detach()}


    # Do I need this?
    def sample(self,
               num_samples:int,
               current_device: int, **kwargs) -> Tensor:
        """
        Samples from the latent space and return the corresponding
        image space map.
        :param num_samples: (Int) Number of samples
        :param current_device: (Int) Device to run the model
        :return: (Tensor)
        """
        z = torch.randn(num_samples,
                        self.latent_dim)

        z = z.to(current_device)

        samples = self.decode(z)
        return samples

    # Do I need this?
    def generate(self, x: Tensor, **kwargs) -> Tensor:
        """
        Given an input image x, returns the reconstructed image
        :param x: (Tensor) [B x C x H x W]
        :return: (Tensor) [B x C x H x W]
        """

        return self.forward(x)[0]

import numpy as np
from UTILS.tensor_ops import _2tensor, cfg
cfg.device_ = 'cuda:0'
cfg.use_float64_ = False
cfg.init = True

input_dim = 10
my_data_sample_x = (np.random.rand(1000,10)-0.5)*2 + 5 # 0.75~1.25
my_data_sample_y = my_data_sample_x.mean(-1)**2 + my_data_sample_x.mean(-1)*(-2) + 1

# solve the weight inital problem
vae_mod = VanillaVAE(input_dim=input_dim, latent_dim=16, hidden_dim=32, degenerate2ae=False)
my_data_sample_x = _2tensor(my_data_sample_x)
vae_mod = _2tensor(vae_mod)
from torch.optim import Adam
optimizer = Adam(vae_mod.parameters(), lr=3e-3)

print("Start training VAE...")
vae_mod.train()

for epoch in range(5000):
    overall_loss = 0
    x = my_data_sample_x

    optimizer.zero_grad()

    #  [self.decode(z), input, mu, log_var]    #  x_hat, mean, log_var
    x_hat, x_origin, mean, log_var = vae_mod(x) # model(x)
    error = ( torch.abs(x_hat - x) )/( torch.abs(x)+1e-9 )
    std_, mean_ = torch.std_mean(error, unbiased=False)
    print('\test error mean %.2f%% and std %.2f'%(mean_.item()*100, std_.item()) , end='\t')


    lossdict = vae_mod.loss_function(x=x, x_hat=x_hat, mean=mean, log_var=log_var, kld_loss_weight=1, recons_loss_weight=1)
    loss = lossdict['loss']
    overall_loss += loss.item()

    loss.backward()
    optimizer.step()
    # 'Reconstruction_Loss':recons_loss.detach(), 'KLD'
    print("\tEpoch", epoch + 1, "complete!", "\tReconstruction_Loss: ", lossdict['Reconstruction_Loss'].item(), "\tKLD: ", lossdict['KLD'].item())

print("Finish!!")